<a target="_blank" href="https://colab.research.google.com/github/greatakela/GenChatBot/blob/main/Notebooks/GNLP_HW2_FLAN_T5_train_model.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [5]:
data_path = '/content/drive/MyDrive/GNLP/HW2/'

In [6]:
import gc
import torch
import os

In [7]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
cleanup()

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import wandb
#wandb.login(key=userdata.get('wandb'))
wandb.login()
#wandb.init(project="gen_chatbot_models")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bagor123 (greatakela) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#Обучение модели FLAN-T5

In [7]:
%pip install -q -U datasets matplotlib tqdm evaluate rouge-score bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   

In [9]:
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")

In [10]:
spock_lines = pd.read_pickle(data_path+"spock_lines_context.pkl")
spock_lines

,QUESTION,ANSWER,CONTEXT
0,"All operating, sir.",It can't be the screen then. Definitely someth...,Check the circuit.
1,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,I have a fix. It comes from the Talos star group.
2,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"A ship in trouble making a forced landing, sir..."
3,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,They were keyed to cause interference and attr...
4,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"It's a radio wave, sir. We're passing through ..."
...,...,...,...
38019,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,One of us should be able to get to him. I keep...
38020,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,"I'm afraid that is a forlorn hope, Doctor. The..."
38021,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,"Yes, well what worries me most is that Jim see..."
38022,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,Good. Knowledge of the captain's aberrant beha...


In [11]:
spock_lines["COMBINED"] = spock_lines[["QUESTION", "CONTEXT"]].apply(
    lambda spock_lines: "context: " + spock_lines["CONTEXT"] + "</s>" + 'question: '
+ spock_lines['QUESTION'], axis = 1)
spock_lines

,QUESTION,ANSWER,CONTEXT,COMBINED
0,"All operating, sir.",It can't be the screen then. Definitely someth...,Check the circuit.,context: Check the circuit.</s>question: All o...
1,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,I have a fix. It comes from the Talos star group.,context: I have a fix. It comes from the Talos...
2,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"A ship in trouble making a forced landing, sir...",context: A ship in trouble making a forced lan...
3,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,They were keyed to cause interference and attr...,context: They were keyed to cause interference...
4,We've no ships or Earth colonies that far out.,Their call letters check with a survey expedit...,"It's a radio wave, sir. We're passing through ...","context: It's a radio wave, sir. We're passing..."
...,...,...,...,...
38019,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,One of us should be able to get to him. I keep...,context: One of us should be able to get to hi...
38020,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,"I'm afraid that is a forlorn hope, Doctor. The...","context: I'm afraid that is a forlorn hope, Do..."
38021,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,"Yes, well what worries me most is that Jim see...","context: Yes, well what worries me most is tha..."
38022,"Well, then, so be it. Our responsibility is cl...",Whatever happened to the captain on that plane...,Good. Knowledge of the captain's aberrant beha...,context: Good. Knowledge of the captain's aber...


In [12]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(spock_lines, test_size=0.05, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.shape, val_df.shape

((36122, 4), (1902, 4))

In [13]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
        "valid": Dataset.from_pandas(val_df.reset_index(drop=True)),
    }
)
dataset

DatasetDict({
    train: Dataset({
        features: ['QUESTION', 'ANSWER', 'CONTEXT', 'COMBINED'],
        num_rows: 36122
    })
    valid: Dataset({
        features: ['QUESTION', 'ANSWER', 'CONTEXT', 'COMBINED'],
        num_rows: 1902
    })
})

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [15]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["valid"]]).map(
    lambda x: tokenizer(x["COMBINED"], truncation=True),
    batched=True,
    remove_columns=["CONTEXT", "QUESTION", "ANSWER", "COMBINED"],
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["valid"]]).map(
    lambda x: tokenizer(x["ANSWER"], truncation=True),
    batched=True,
    remove_columns=["CONTEXT", "QUESTION", "ANSWER", "COMBINED"],
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/38024 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/38024 [00:00<?, ? examples/s]

Max target length: 174


In [ ]:
def preprocess_function(sample, padding="max_length"):

    model_inputs = tokenizer(
        sample['COMBINED'], max_length=max_source_length, padding=padding, truncation=True
    )

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(
        text_target=sample["ANSWER"],
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.

    if padding == "max_length":
      labels["input_ids"] = [
          [(l if l != tokenizer.pad_token_id else -100) for l in label]
          for label in labels["input_ids"]
      ]


    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["CONTEXT", "QUESTION", "ANSWER", "COMBINED"]
)
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/36122 [00:00<?, ? examples/s]

Map:   0%|          | 0/1902 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [17]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id = "google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [18]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")

# Metric
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")


# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
##
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
##
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bertscore_result = bertscore.compute(predictions=decoded_preds, references=decoded_labels,lang="en")


    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    rouge_result = rouge.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    rouge_result = {k: round(v * 100, 4) for k, v in rouge_result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    rouge_result["gen_len"] = np.mean(prediction_lens)

    result = {'rouge_1': round(rouge_result['rouge1'], 4),
              'rouge_2': round(rouge_result['rouge2'], 4),
              'rouge_L': round(rouge_result['rougeL'], 4),
              'avg_len': round(rouge_result['gen_len'], 4),
              'bertscore_prec': round(np.mean(bertscore_result['precision']), 4),
              'bertscore_rec': round(np.mean(bertscore_result['recall']), 4),
              'bertscore_f1': round(np.mean(bertscore_result['f1']), 4)}
    return result

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, label_pad_token_id=label_pad_token_id, pad_to_multiple_of=8
)

In [ ]:
import wandb
os.environ["WANDB_PROJECT"] = "gen_chatbot_models"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [23]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-gen-chat_base-5"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    warmup_ratio=0.1,
    weight_decay=0.1,
    optim="adamw_torch",
    # logging & evaluation strategies
    evaluation_strategy="steps",
    eval_steps=400,
    save_strategy="no",
    save_total_limit=1,
    report_to="wandb",
    logging_steps=200,
    push_to_hub=False,
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Rouge 1,Rouge 2,Rouge L,Avg Len,Bertscore Prec,Bertscore Rec,Bertscore F1
400,3.602000,3.304259,6.129600,0.390600,5.628400,13.526300,0.850500,0.847800,0.848800
800,3.379900,3.082215,6.694900,0.483500,6.216000,15.491600,0.857000,0.850700,0.853500
1200,3.171100,2.882498,6.284000,0.451900,5.996400,16.621500,0.862500,0.851300,0.856600
1600,3.029800,2.707834,6.731900,0.565900,6.333900,16.933800,0.860200,0.852700,0.856200
2000,2.872200,2.561545,7.943800,0.985900,7.426200,15.139300,0.862200,0.853800,0.857700
2400,2.739800,2.429126,8.371700,0.924300,7.747400,14.959500,0.860100,0.854000,0.856800
2800,2.619600,2.305211,8.845800,1.117000,8.248200,14.536800,0.861600,0.854400,0.857700
3200,2.507600,2.199834,9.136900,1.258800,8.532200,14.293900,0.862600,0.855000,0.858500
3600,2.456700,2.105318,9.666300,1.442400,8.970200,12.928000,0.863400,0.854700,0.858700
4000,2.366500,2.006496,9.771800,1.430200,9.043800,13.238700,0.862200,0.855400,0.858500


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainOutput(global_step=11290, training_loss=2.2684395545979745, metrics={'train_runtime': 10245.3337, 'train_samples_per_second': 17.629, 'train_steps_per_second': 1.102, 'total_flos': 1.2367402542563328e+17, 'train_loss': 2.2684395545979745, 'epoch': 5.0})

In [25]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/greatakela/flan-t5-base-gen-chat_base-5/commit/9af78c1bf8760c8b53185a7e3d001f146d350d92', commit_message='End of training', commit_description='', oid='9af78c1bf8760c8b53185a7e3d001f146d350d92', pr_url=None, repo_url=RepoUrl('https://huggingface.co/greatakela/flan-t5-base-gen-chat_base-5', endpoint='https://huggingface.co', repo_type='model', repo_id='greatakela/flan-t5-base-gen-chat_base-5'), pr_revision=None, pr_num=None)

In [26]:
wandb.finish()

eval/avg_len,▃▆██▅▅▅▄▂▃▂▁▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▁
eval/bertscore_f1,▁▄▅▅▆▅▆▆▆▆▆▆▆▆▇▇▇▇▇█████████
eval/bertscore_prec,▁▄▇▆▇▆▆▇▇▇▇▆▆▇▇▇▇█▇█████████
eval/bertscore_rec,▁▃▃▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇██████████
eval/loss,█▇▇▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
eval/rouge_1,▁▂▁▂▃▃▄▄▄▄▅▅▅▆▆▇▆▇▇▇▇███▇███
eval/rouge_2,▁▁▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇▇▇███
eval/rouge_L,▁▂▁▂▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇███▇███
eval/runtime,█▁▁▁▁▁▂▂▂▁▃▃▂▂▂▂▃▂▃▂▃▁▁▂▃▃▃▃
eval/samples_per_second,▁█████▇▇▇█▆▆▇▆▆▆▆▆▆▇▆██▇▆▆▆▆
eval/steps_per_second,▁█████▇▇▇█▆▆▇▆▆▆▆▆▆▇▆██▇▆▆▆▆
